# 2018-10-02 (martes)

### Presentaciones

### Tarea 3

---


### Antes de partir

Primero, cargamos informacion relevante para la clase:


In [1]:
#conexion 1: leer archivo sqlite3

#cargo librerias
import pandas as pd
import sqlite3

#cargamos funcion "display", para ver los DataFrame de pandas formateados
from IPython.display import display

#leo los datos
db_path = 'data_sql.db'

#creo conexion
db_conn = sqlite3.connect(db_path)


In [2]:
#cargo datos del servidor

#en este ejemplo, el servidor de datos es MySQL, por lo que usamos 
#libreria pymysql para administrar la conexion
import pymysql.cursors

#datos de conexion al servidor (muy similar a lo que es un "path")
host = '190.107.176.7'
user = 'reportej_alum'
passwd = 'clave_alumno2018'
database = 'reportej_eae253b'

# Creamos una conexion a la base de datos
casen_connection = pymysql.connect(host=host,
                             user=user,
                             password=passwd,
                             db=database,)

### Ejemplo 1. ¿cómo saber las tablas que hay en la base de datos?

- SQLite3: SELECT * FROM sqlite_master;

- pyMySQL: SHOW TABLES


In [3]:
#lista de tablas de base de datos en sqlite3
query = '''
    SELECT * FROM sqlite_master;
'''

df_cuenta = pd.read_sql(query, db_conn)

display(df_cuenta)

,type,name,tbl_name,rootpage,sql
0,table,mis_datos,mis_datos,2,"CREATE TABLE ""mis_datos"" (\n""index"" INTEGER,\n..."
1,index,ix_mis_datos_index,mis_datos,3,"CREATE INDEX ""ix_mis_datos_index""ON ""mis_datos..."
2,table,tabla_nombres,tabla_nombres,4,"CREATE TABLE ""tabla_nombres"" (\n""index"" INTEGE..."
3,index,ix_tabla_nombres_index,tabla_nombres,5,"CREATE INDEX ""ix_tabla_nombres_index""ON ""tabla..."


In [4]:
#lista de tablas de base de datos en mysql

query = '''
    SHOW TABLES
'''

df_cuenta = pd.read_sql(query, casen_connection)

display(df_cuenta)

,Tables_in_reportej_eae253b
0,casen2015
1,casen2017


### Ejemplo 1: informacion basica de encuesta Casen

¿qué hace la siguiente query?

In [5]:
query = '''
    select region, count(*)
    from casen2015
    group by region
'''

df_cuenta = pd.read_sql(query, casen_connection)

display(df_cuenta)

,region,count(*)
0,1,8458
1,2,6655
2,3,13819
3,4,12464
4,5,27998
5,6,22805
6,7,17330
7,8,36098
8,9,22042
9,10,19386


##### Ejercicio: calcule el número de personas por región, usando el factor de expansión regional (expr)

In [6]:
query = '''

'''

df_cuenta = pd.read_sql(query, casen_connection)

display(df_cuenta)

,region,sum(expr)
0,1,326966.0
1,2,574413.0
2,3,281153.0
3,4,758108.0
4,5,1824098.0
5,6,914682.0
6,7,1032683.0
7,8,2067335.0
8,9,983653.0
9,10,863439.0


### Ejemplo 2: SQL Injection

In [8]:
#calcule el número de personas de la región metropolitana

sql_query = '''
    SELECT region, sum(expr) as personas
    FROM casen2017
    WHERE region = 13
    GROUP BY region
'''

data = pd.read_sql(sql_query, casen_connection)

display(data)

,region,personas
0,13.0,7223236.0


"SQL injection" nos permite ingresar parámetros desde "afuera" de SQL:

In [ ]:
sql_query = '''
    SELECT region, count(*)
    FROM casen2017
    WHERE region = %s
    GROUP BY region
'''

region_selected = 13

#, params=[region_selected,]
data = pd.read_sql(sql_query, casen_connection, params=(13,))

display(data)

### Ejemplo 3: usar Subqueries

¿qué hace la siguiente query?

In [ ]:
query = '''
    SELECT *
    FROM (
        select region, count(*) as poblacion
        from casen2015
        group by region
    )
    WHERE POBLACION > 1000000
'''

df_cuenta = pd.read_sql(query, casen_connection)

display(df_cuenta)

### Otras operaciones: crear variables en SQL

Calcule el ingreso promedio de la región (usar variable ytot)

In [12]:
query = '''
    SELECT region, sumay/poblacion as y_prom, promy
    FROM (
        select region, count(*) as poblacion, sum(ytot) as sumay, avg(ytot) as promy
        from casen2015
        group by region
    ) as a

'''

df_cuenta = pd.read_sql(query, casen_connection)

display(df_cuenta)

,region,y_prom,promy
0,1,236046.367309,400980.151576
1,2,271448.659232,491963.732894
2,3,260854.067121,416974.245639
3,4,202797.761068,316314.765855
4,5,218433.590050,339440.731211
5,6,201514.585454,308322.047722
6,7,180604.489262,249293.173947
7,8,178191.565013,262877.890957
8,9,172216.631991,236569.799473
9,10,185758.990548,260592.212951


In [ ]:
Ahora, usando factor de expansión regional:

### Otras operaciones: unir bases de datos ("append" de Stata, o "concatenate" de pandas)

Para hacer esto, en SQL existe la sentencia "UNION". Existen dos modalidades comunmente
- UNION
- UNION ALL: no elimina filas duplicadas

### Ejercicio: ¿cómo obtendríamos el número de personas por región, para cada Casen?